In [1]:
from models.densenet121 import DenseNet121
from constant import *
from utils import *
import torch
import torchvision
from models.densenet import DenseNet
from torch.autograd import Variable
import numpy as np
import os
import shutil

In [2]:
def convert(model_name):
    # get old network
    from_model = '%s/densenet121/%s/model.path.tar' % (MODEL_DIR, model_name)
    from_net = DenseNet121(14)
    parallel_net = torch.nn.DataParallel(from_net, device_ids=[0]).cuda()
    checkpoint = torch.load(from_model)
    parallel_net.load_state_dict(checkpoint['state_dict'])
    
    # create folder
    to_path = '%s/densenet/densenet121/%s' % (MODEL_DIR, model_name)
    to_model = '%s/model.path.tar' % to_path
    if not os.path.isdir(to_path):
        os.makedirs(to_path)
        
    # convert to new
    to_net = DenseNet('densenet121')
    to_net.features = from_net.densenet121.features
    to_net.classifier = from_net.densenet121.classifier
    checkpoint['state_dict'] = to_net.state_dict()
    torch.save(checkpoint, to_model)
    
    # move stat
    from_stat = '%s/densenet121/%s/stat.h5' % (LOG_DIR, model_name)
    to_path = '%s/densenet/densenet121/%s' % (LOG_DIR, model_name)
    to_stat = '%s/stat.h5' % to_path
    if not os.path.isdir(to_path):
        os.makedirs(to_path)
    try:
        shutil.copyfile(from_stat, to_stat)
    except:
        print('model', model_name)

In [3]:
model_names = [
#     '20180420-092017',
#     '20180422-075022',
#     '20180426-065637',
#     '20180427-091900',
#     '20180427-230838',
#     '20180428-111638',
#     '20180429-131436',
#     '20180429-131732',
#     '20180429-195702',
#     '20180429-200046',
#     '20180429-200210',
#     '20180430-113040',
#     '20180430-162711',
#     '20180501-055454',
#     '20180501-055908',
#     '20180430-213936',
#     '20180430-215023',
#     '20180430-214452'
    '20180429-130928' # best densenet121 model
]
for model_name in model_names:
    convert(model_name)

In [6]:
# test
model_name = '20180429-130928'
to_path = '%s/densenet/densenet121/%s' % (MODEL_DIR, model_name)
to_model = '%s/model.path.tar' % to_path
test_net = DenseNet('densenet121')
test_loader = test_dataloader(test_net, image_list_file=CHEXNET_TEST_CSV, percentage=1, agumented=True)
test_checkpoint = torch.load(to_model)
test_net.load_state_dict(test_checkpoint['state_dict'])
test_net = torch.nn.DataParallel(test_net, device_ids=[0]).cuda()

In [7]:
def test(model, dataloader, agumented=False):
    model.eval()
    targets = torch.FloatTensor()
    targets = targets.cuda()
    preds = torch.FloatTensor()
    preds = preds.cuda()
    
    for data, target in dataloader:
        target = target.cuda()
        if agumented:
            bs, cs, c, h, w = data.size()
            data = data.view(-1, c, h, w)
        data = Variable(data.cuda(), volatile=True)
        pred = model(data)
        if agumented:
            pred = pred.view(bs, cs, -1).mean(1)
        targets = torch.cat((targets, target), 0)
        preds = torch.cat((preds, pred.data), 0)
    aurocs = compute_aucs(targets, preds)
    aurocs_avg = np.array(aurocs).mean()
    
    print('The average AUROC is {0:.3f}'.format(aurocs_avg))
    for i in range(N_CLASSES):
        print('The AUROC of {} is {}'.format(CLASS_NAMES[i], aurocs[i]))

In [8]:
test(test_net, test_loader, True)

The average AUROC is 0.849
The AUROC of Atelectasis is 0.8283099353189022
The AUROC of Cardiomegaly is 0.9177011251303541
The AUROC of Effusion is 0.8851548655853833
The AUROC of Infiltration is 0.7112059234984651
The AUROC of Mass is 0.8659420956453638
The AUROC of Nodule is 0.7885605536921866
The AUROC of Pneumonia is 0.7739571101529881
The AUROC of Pneumothorax is 0.8874663477489905
The AUROC of Consolidation is 0.8162082170702861
The AUROC of Edema is 0.8971113097712183
The AUROC of Emphysema is 0.9271256410339128
The AUROC of Fibrosis is 0.8408021976288977
The AUROC of Pleural Thickening is 0.7873279987663767
The AUROC of Hernia is 0.957229839369985
